# Building event features

> These are Go notebooks: In order to use the GoNB Jupyter Kernel, please install GoNB from here: https://github.com/janpfeifer/gonb

Note also that for local package development, you can put: `!*go mod edit -replace "github.com/umbralcalc/trywizard=/path/to/trywizard"` at the top of any cell.

In this notebook we're building event features from local csv files which aggregate information over all the data sources listed in the `README.md`.

In [1]:
!*go mod edit -replace "github.com/umbralcalc/trywizard=/home/robert/Code/trywizard"

In [ ]:
import (
	"os"
	"github.com/umbralcalc/trywizard/pkg/features"
	"github.com/go-gota/gota/series"
	"github.com/go-gota/gota/dataframe"
	"github.com/go-echarts/go-echarts/v2/charts"
	"github.com/go-echarts/go-echarts/v2/opts"
	gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

%%

scatter := charts.NewScatter()
scatter.SetGlobalOptions(
	charts.WithXAxisOpts(opts.XAxis{
		Name: "minute",
	}),
	charts.WithTooltipOpts(opts.Tooltip{
		Trigger:   "item",
		Formatter: "({c})",
	}),
)

df := features.GetRawEventsDataFrame()
cdf := features.TransformToEventCounts(&df)
for _, eventType := range features.GetUniqueEventTypes(&df) {
	points := make([]opts.ScatterData, 0)
	fdf := cdf.Filter(dataframe.F{
		Colname:    "event_type",
		Comparator: series.Eq,
		Comparando: eventType,
	})
	minutes, _ := fdf.Col("minute").Int()
	counts := fdf.Col("event_type_COUNT").Float()
	for i, minute := range minutes {
		points = append(points, opts.ScatterData{
			Value: []interface{}{minute, counts[i]},
		})
	}

	scatter.AddSeries(eventType, points)
}

gonb_echarts.Display(scatter, "width: 1024px; height:400px; background: white;")